In [3]:
# Explanation of prediction output when activation is sigmoid:
# https://forum.freecodecamp.org/t/model-predict-output/470349

import tensorflow as tf
import tensorflow_datasets as tfds
import logging, os
import pandas as pd
import time
from sklearn import metrics
import numpy as np

BASE_PATH = "../../../../../local_data/practice/tfds/"
DATA_PATH = "../../../../../local_data/tfds/"
OUTPUT_PATH = BASE_PATH+"predict_example_01/"
os.system("mkdir -p " + OUTPUT_PATH)


# Load the dataset
(train_dataset, test_dataset), metadata = tfds.load(
    'cats_vs_dogs',
    data_dir=DATA_PATH,
    split=['train[:80%]', 'train[80%:]'],
    # split=['train[:80%]', 'train[99%:]'],
    with_info=True,
    as_supervised=True
)

# Preprocess the data
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [256, 256])
    image = image / 255.0
    return image, label

train_dataset = train_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

batch_size = 32
train_dataset = train_dataset.cache().batch(batch_size).prefetch(buffer_size=10)
test_dataset = test_dataset.cache().batch(batch_size).prefetch(buffer_size=10)


# Apply data augmentation
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.1)
    return image, label

train_dataset = train_dataset.map(augment)


# Model prediction
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [4]:
# Train the model
epochs=1
model.fit(train_dataset, epochs=epochs)


# Make predictions
predictions = model.predict(test_dataset)
allpreds=predictions.flatten()
allpnorms = np.where(allpreds > 0.5, 1, 0)

alllabels=np.empty(0)
for images, labels in test_dataset:
    alllabels = np.append(alllabels, labels.numpy().flatten())

score = metrics.accuracy_score(alllabels, allpnorms)
print("test dataset accuracy score: {}".format(score))

collabels = pd.DataFrame(alllabels, columns=["l"])
colpreds = pd.DataFrame( allpreds, columns=["pred"])
pnorm = pd.DataFrame( allpnorms, columns=["pnorm"])
diff = collabels["l"] - pnorm["pnorm"]

compare = pd.concat([collabels, colpreds,pnorm,diff], axis=1)
compare.columns = ["l", "pred", "pnorm","diff"]
print(compare)

compare.to_csv(OUTPUT_PATH + "pred_test_fit.csv", index=False)    

# Save model
timestr = time.strftime("%Y%m%d-%H%M%S")
filename = f"acc_{score:.3f}_epochs_{epochs:.3f}_date_{timestr}.h5"
fullpath = f"{OUTPUT_PATH}{filename}"
print("Saving model to ", filename)
model.save(fullpath)

177/582 [========>.....................] - ETA: 19s - loss: 0.9084 - accuracy: 0.5980

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9


208/582 [=========>....................] - ETA: 18s - loss: 0.8685 - accuracy: 0.6035

219/582 [==========>...................] - ETA: 17s - loss: 0.8570 - accuracy: 0.6056

Corrupt JPEG data: 396 extraneous bytes before marker 0xd9


...

Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


 87/146 [================>.............] - ETA: 0s

Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9


146/146 [==============================] - 2s 10ms/step
test dataset accuracy score: 0.7308684436801376
        l      pred  pnorm  diff
0     1.0  0.557551      1   0.0
1     0.0  0.516746      1  -1.0
2     1.0  0.851982      1   0.0
3     1.0  0.720640      1   0.0
4     1.0  0.804592      1   0.0
...   ...       ...    ...   ...
4647  0.0  0.243625      0   0.0
4648  0.0  0.669425      1  -1.0
4649  0.0  0.414941      0   0.0
4650  1.0  0.604247      1   0.0
4651  0.0  0.371276      0   0.0

[4652 rows x 4 columns]
Saving model to  acc_0.731_epochs_1.000_date_20250718-175545.h5


/media/data/miniconda3/envs/jh_class/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
